# Model Sharing and Saving

## Sci-Kit Learn

In [2]:
from sklearn import svm
from sklearn import datasets

clf = svm.SVC(gamma='scale')
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)  


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [3]:
import pickle
s = pickle.dumps(clf)

In [4]:
clf2 = pickle.loads(s)
clf2.predict(X[0:1])

y[0]


0

### With joblib

In [6]:
from sklearn.externals import joblib

# Saving a model
joblib.dump(clf, 'clf_model.pkl')

# Loading a model
loaded_model = joblib.load('clf_model.pkl')

## Keras

In [36]:
import os

* ResNet50 CNN Model from Keras...
* (weights are 100MB!)

In [3]:
import json
from keras.models import model_from_json, load_model
from keras.applications import resnet50

model = resnet50.ResNet50(include_top=True, weights='imagenet')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [35]:
model.save('model.h5')
new_model_2 = load_model('model.h5')

In [47]:
print(
    f"{os.path.getsize('model.h5') / 1024 ** 2 : .1f} MB"
)

 98.3 MB


In [4]:
model.save_weights('model_weights.h5')

with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())


In [5]:
with open('model_architecture.json', 'r') as f:
    new_model_1 = model_from_json(f.read())
    
new_model_1.load_weights('model_weights.h5')


In [31]:

#os.unlink('model_weights.h5')
#os.unlink('model_architecture.json')
#os.unlink('model.h5')

# Onnx

In [33]:
!pip install -q onnx onnxruntime

## ONNX: Keras

In [50]:
!pip install -q onnxmltools

In [52]:
import onnxmltools 

# Update the input name and path for your Keras model
input_keras_model = 'model.h5'

# Change this path to the output name and path for the ONNX model
output_onnx_model = 'model.onnx'

# Load your Keras model
keras_model = load_model(input_keras_model)

# Convert the Keras model into ONNX
onnx_model = onnxmltools.convert_keras(model)

# Save as protobuf
onnxmltools.utils.save_model(onnx_model, output_onnx_model)

I0826 12:32:48.963560 139650112431936 tfonnx.py:475] Using tensorflow=1.14.0, onnx=1.5.0, tf2onnx=1.5.2/0c735a
I0826 12:32:48.965300 139650112431936 tfonnx.py:478] Using opset <onnx, 10>


In [53]:
print(
    f"{os.path.getsize('model.onnx') / 1024 ** 2 : .1f} MB"
)

 97.8 MB


## ONNX: SciKit

In [16]:
!pip install -q skl2onnx

In [12]:
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

clr = RandomForestClassifier(n_estimators=50)
clr.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
# Convert into ONNX format with onnxmltools
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([1, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)

with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())


W0826 12:23:35.240221 139650112431936 _topology.py:1094] The maximum opset needed by this model is only 9.
W0826 12:23:35.241531 139650112431936 _topology.py:1094] The maximum opset needed by this model is only 1.


In [28]:
#Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy

sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]

In [29]:
pred_onx

array([0, 0, 2, 1, 1, 2, 0, 2, 2, 1, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0,
       1, 0, 0, 2, 0, 0, 0, 0, 1, 1, 2, 0, 2, 2, 1, 2], dtype=int64)

# Extra

## Tensorflow sub-Model Saving

In [ ]:
import tensorflow as tf

# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
    sess.run(init_op)
    # Do some work with the model.
    inc_v1.op.run()
    dec_v2.op.run()
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in path: %s" % save_path)

In [ ]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    # Check the values of the variables
    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())